In [60]:
%matplotlib qt

import time

import matplotlib.pyplot as plt

import numpy as np
import exafmm.laplace as laplace

from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_p2p_parallel
from fmm.surface import compute_surface, scale_surface

In [2]:
def timer(f, *args, **kwargs):
    s = time.time()
    res = f(*args, **kwargs)
    print(f'Runtime: {time.time()-s:.2f} s')
    return res

In [43]:
exp = Fmm('test_config')
timer(exp.run)

Runtime: 0.88 s


In [44]:
# Set same sources, targets and source densities
src_coords = exp.sources
trg_coords = exp.targets
src_charges = exp.source_densities

In [45]:
# create a list of source instances
sources = laplace.init_sources(src_coords, src_charges)

# create a list of target instances
targets = laplace.init_targets(trg_coords)

In [46]:
fmm = laplace.LaplaceFmm(p=5, ncrit=exp.config['max_points'], filename="test_file.dat")

In [47]:
tree = laplace.setup(sources, targets, fmm)

In [48]:
trg_values = timer(laplace.evaluate, tree, fmm)

Runtime: 0.31 s


In [49]:
exp.target_potentials

array([1109.8365, 1013.8969, 1137.1888, ..., 1033.253 , 1073.9423,
       1096.0267], dtype=float32)

In [50]:
trg_values[:, 0]

array([1110.05684806, 1013.58162298, 1137.50210119, ..., 1032.98995011,
       1074.04157202, 1096.24109527])

In [51]:
np.allclose(exp.target_potentials, trg_values[:, 0], rtol=0.002, atol=0)

True

In [74]:
py_multipole_expansion = exp.multipole_expansions[
    exp.key_to_index[0]:exp.key_to_index[0]+exp.nequivalent_points
]
ex_multipole_expansion = np.array(tree.nodes[0].up_equiv)

upward_equivalent_surface = scale_surface(
    surf=exp.equivalent_surface,
    radius=exp.r0,
    level=0, 
    center=exp.x0, 
    alpha=exp.alpha_inner
)

target = np.array([[-10,10,10]])

py_result = laplace_p2p_serial(
    sources=upward_equivalent_surface, 
    targets=target, 
    source_densities=py_multipole_expansion
)

ex_result = laplace_p2p_serial(
    sources=upward_equivalent_surface, 
    targets=target, 
    source_densities=ex_multipole_expansion
)

In [75]:
ex_result

array([46.65264], dtype=float32)

In [76]:
py_result

array([46.65678], dtype=float32)